<a href="https://colab.research.google.com/github/nguyenkimthach/Optimization_Binary_Neural_Network/blob/main/CNN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml h5py

In [ ]:
# import the necessary packages
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras.utils.vis_utils import plot_model
import time

from keras.layers import Dropout
from keras import backend as K
from keras.models import load_model
from keras.datasets import mnist
from keras.utils import np_utils
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)

In [ ]:
# load minit data
(X_train, y_train),(X_test, y_test) = mnist.load_data()

#printng shape of mnist train and test data
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
'''
CNN accepts a specific format when using tensorflow e.g.(batch, height, width, channels)
As all the images are in grayscale, the number of channels is 1
'''
# reshape the data to four dimensions, due to the input of model
# reshape to be [samples/batch][width][height][pixels/channels]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
print(X_train.shape )
print(X_test.shape )

'''
rescaled the image data below so that each pixel lies in the interval [0, 1] instead of [0, 255].
'''
# Normalization
X_train /= 255.0
X_test /= 255.0


In [ ]:
#one-hot encoding an integer is converted to an array which contains only
#one ‘1’ and the rest elements are ‘0’.

# one-hot
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(y_train.shape )
print(y_test.shape )
num_classes = y_test.shape[1]

In [ ]:
model = Sequential()

# filters 32, size of filters (3,3)
model.add(Conv2D(16, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1)) #BatchNormalization normalizes the matrix after it is been through a convolution layer so that the scale of each dimension remains the same
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) # act as an input to the Dense layers.

# Fully connected layer
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax')) # Softmax activation enables us to calculate the output based on the probabilities

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

start_time = time.time()
trained_model=model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(X_test, y_test),
    shuffle=True
)
print("execution time:  %s ms" % ((time.time() - start_time)*1000))


test_loss, test_acc = model.evaluate(X_test, y_test)

In [ ]:
for layer in model.layers:
    print(layer.name, layer)
#save mode
model.save('my_nom1_model.h5')

In [ ]:
#epoch_draw = [0, 5, 10, 15, 20, 25, 30]
#accuracy_draw = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
#plt.xticks( np.arange(7),epoch_draw )
#plt.yticks([0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96,0.97,0.98,0.99, 1])
plt.plot(trained_model.history['accuracy'])
plt.plot(trained_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.max(trained_model.history['accuracy']))

In [ ]:
plt.plot(trained_model.history['loss'])
plt.plot(trained_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.min(trained_model.history['loss']))
print(np.min(trained_model.history['val_loss']))

In [ ]:
print(f"Test accuracy {test_acc * 100:.2f} %")